In [1]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
import json
import nltk
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from tqdm import tqdm
smoothie = SmoothingFunction().method4
rouge = evaluate.load('rouge')
messages_final = json.load(open('data/messages_final.json'))
messages_boh_original_final = json.load(open('data/messages_boh_original_final.json'))
messages_no_sw_original_final = json.load(open('data/messages_no_sw_original_final.json'))
test_data = json.load(open('data/test_data.json'))

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_16_2/checkpoint-50/"
torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, device_map="auto")

In [8]:
len(test_data)

66

In [9]:
len(messages_final[0])

22

In [10]:
from datasets import load_dataset

dataset = load_dataset("thu-coai/esconv")

In [19]:

bleu_2_final, bleu_2_boh_original_final, bleu_2_no_sw_original_final = [], [], []
bleu_4_final, bleu_4_boh_original_final, bleu_4_no_sw_original_final = [], [], []
bert_score_final, bert_score_boh_original_final, bert_score_no_sw_original_final = [], [], []
rouge_final, rouge_boh_original_final, rouge_no_sw_original_final = [], [], []
meteor_final, meteor_boh_original_final, meteor_no_sw_original_final = [], [], []

for j in tqdm(range(3, 53)):
    data = json.loads(dataset['test'][j]['text'])
    data = data['dialog']
    j = j-3
    try:
        for i in range(0, len(data),1):
            #print(i)
            if data[i+1]['speaker'] == "sys":
                reference = data[i+1]['text']
            elif data[i+2]['speaker'] == "sys":
                reference = data[i+1]['text']
            elif data[i+3]['speaker'] == "sys":
                reference = data[i+1]['text']
            #print(messages_final[j][i]['content'])
            
            bleu_2_final.append(sentence_bleu(messages_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_2_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_2_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], reference, weights=(0.5,0.5), smoothing_function=smoothie))
            bleu_4_final.append(sentence_bleu(messages_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bleu_4_boh_original_final.append(sentence_bleu(messages_boh_original_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bleu_4_no_sw_original_final.append(sentence_bleu(messages_no_sw_original_final[j][i]['content'], reference, weights=(0.25,0.25,0.25,0.25), smoothing_function=smoothie))
            bert_score_final.append(score([messages_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            bert_score_boh_original_final.append(score([messages_boh_original_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            bert_score_no_sw_original_final.append(score([messages_no_sw_original_final[j][i]['content']], [reference], lang='en', model_type='bert-base-uncased'))
            rouge_final.append(rouge.compute(predictions=[messages_final[j][i]['content']], references=[reference]))
            rouge_boh_original_final.append(rouge.compute(predictions=[messages_boh_original_final[j][i]['content']], references=[reference]))
            rouge_no_sw_original_final.append(rouge.compute(predictions=[messages_no_sw_original_final[j][i]['content']], references=[reference]))
            meteor_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_final[j][i]['content'])], word_tokenize(reference))))
            meteor_boh_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_boh_original_final[j][i]['content'])], word_tokenize(reference))))
            meteor_no_sw_original_final.append((nltk.translate.meteor_score.meteor_score([word_tokenize(messages_no_sw_original_final[j][i]['content'])], word_tokenize(reference))))
            
    except:
        print(j,i)

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:31<25:53, 31.70s/it]

0 22
1 0


  6%|▌         | 3/50 [01:39<26:13, 33.49s/it]

2 52


...

34 27
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0


 88%|████████▊ | 44/50 [14:03<00:46,  7.78s/it]

43 24
44 0
45 0


100%|██████████| 50/50 [15:23<00:00, 18.47s/it]

46 65
47 0
48 0
49 0


In [20]:
rougel_final = [i['rougeL'] for i in rouge_final]
rougel_boh_original_final = [i['rougeL'] for i in rouge_boh_original_final]
rougel_no_sw_original_final = [i['rougeL'] for i in rouge_no_sw_original_final]

In [21]:
bert_p_boh = [i[0].item() for i in bert_score_boh_original_final]
bert_r_boh = [i[1].item() for i in bert_score_boh_original_final]
bert_f1_boh = [i[2].item() for i in bert_score_boh_original_final]

bert_p_no_sw = [i[0].item() for i in bert_score_no_sw_original_final]
bert_r_no_sw = [i[1].item() for i in bert_score_no_sw_original_final]
bert_f1_no_sw = [i[2].item() for i in bert_score_no_sw_original_final]

bert_p = [i[0].item() for i in bert_score_final]
bert_r = [i[1].item() for i in bert_score_final]
bert_f1 = [i[2].item() for i in bert_score_final]

In [22]:
import statistics

print("Bleu 2 Final: ", statistics.mean(bleu_2_final))
print("Bleu 2 Boh Original Final: ", statistics.mean(bleu_2_boh_original_final))
print("Bleu 2 No Sw Original Final: ", statistics.mean(bleu_2_no_sw_original_final))
print("Bleu 4 Final: ", statistics.mean(bleu_4_final))
print("Bleu 4 Boh Original Final: ", statistics.mean(bleu_4_boh_original_final))
print("Bleu 4 No Sw Original Final: ", statistics.mean(bleu_4_no_sw_original_final))
print("Rouge L Final: ", statistics.mean(rougel_final))
print("Rouge L Boh Original Final: ", statistics.mean(rougel_boh_original_final))
print("Rouge L No Sw Original Final: ", statistics.mean(rougel_no_sw_original_final))
print("Bert Precision Final: ", statistics.mean(bert_p))
print("Bert Recall Final: ", statistics.mean(bert_r))
print("Bert F1 Final: ", statistics.mean(bert_f1))
print("Bert Precision Boh Original Final: ", statistics.mean(bert_p_boh))
print("Bert Recall Boh Original Final: ", statistics.mean(bert_r_boh))
print("Bert F1 Boh Original Final: ", statistics.mean(bert_f1_boh))
print("Bert Precision No Sw Original Final: ", statistics.mean(bert_p_no_sw))
print("Bert Recall No Sw Original Final: ", statistics.mean(bert_r_no_sw))
print("Bert F1 No Sw Original Final: ", statistics.mean(bert_f1_no_sw))
print("Meteor Final: ", statistics.mean(meteor_final))
print("Meteor Boh Original Final: ", statistics.mean(meteor_boh_original_final))
print("Meteor No Sw Original Final: ", statistics.mean(meteor_no_sw_original_final))


Bleu 2 Final:  0.04957757090498865
Bleu 2 Boh Original Final:  0.04903233914141645
Bleu 2 No Sw Original Final:  0.04857005628140412
Bleu 4 Final:  0.012798795480772743
Bleu 4 Boh Original Final:  0.012722933710740895
Bleu 4 No Sw Original Final:  0.012669187262289678
Rouge L Final:  0.12630593607153845
Rouge L Boh Original Final:  0.1282707451016453
Rouge L No Sw Original Final:  0.11609526455569318
Bert Precision Final:  0.40962716490872164
Bert Recall Final:  0.4401547888998931
Bert F1 Final:  0.42005367690728884
Bert Precision Boh Original Final:  0.40952830477538754
Bert Recall Boh Original Final:  0.42478142840798294
Bert F1 Boh Original Final:  0.4134836722438955
Bert Precision No Sw Original Final:  0.41169440706318383
Bert Recall No Sw Original Final:  0.41719828202983755
Bert F1 No Sw Original Final:  0.4108598335819729
Meteor Final:  0.1214522739356365
Meteor Boh Original Final:  0.12303861719762525
Meteor No Sw Original Final:  0.12241994402193376


In [24]:
import torch

perplexity_final = torch.load('data/perplexity_final.pt')
perplexity_boh = torch.load('data/perplexity_boh_final.pt')
perplexity_sw = torch.load('data/perplexity_no_sw_final.pt')


In [25]:
perplexity_final1 = []
perplexity_final1.extend(per for sublist in perplexity_final for per in sublist )
perplexity_boh1 = []
perplexity_boh1.extend(per for sublist in perplexity_boh for per in sublist )
perplexity_sw1 = []
perplexity_sw1.extend(per for sublist in perplexity_sw for per in sublist )

print("Perplexity Final: ", torch.mean(torch.tensor(perplexity_final1)))
print("Perplexity Boh Original Final: ", torch.mean(torch.tensor(perplexity_boh1)))
print("Perplexity No Sw Original Final: ", torch.mean(torch.tensor(perplexity_sw1)))

Perplexity Final:  tensor(8.9637)
Perplexity Boh Original Final:  tensor(57.2150)
Perplexity No Sw Original Final:  tensor(55.1800)


In [26]:
import json
import torch
from tqdm import tqdm
import statistics
messages_boh = json.load(open('data/messages_boh_final.json'))
messages_sw = json.load(open('data/messages_no_sw_final.json'))
messages_final = json.load(open('data/messages_final.json'))

In [34]:
messages_boh[2]

[{'role': 'user', 'content': 'My friends trying shame me'},
 {'role': 'assistant',
  'content': 'I here listen It really when friends try shame you Remember you deserve respect kindness from those around you It stand up for yourself communicate your feelings If you need help figuring out how handle situation I here support you'},
 {'role': 'user', 'content': 'How'},
 {'role': 'assistant',
  'content': 'First acknowledge their behavior express your feelings about it For example you say I feel hurt disrespected when you shame me like that Then set boundaries communicate your expectations For instance you say I appreciate spoken to in manner I appreciate if you show me respect Finally offer solution behavior that more For example you suggest Instead of shaming me we discuss our concerns work together find solution'},
 {'role': 'user',
  'content': 'It Christmas my friends say that I donate to charity'},
 {'role': 'assistant',
  'content': 'It sounds like your friends questioning your deci

In [37]:
boh_length, sw_length, final_length = [], [], []
for j in tqdm(range(0,47)):
    for i in range(1,len(messages_final[j]),2):
        final_length.append(len(tokenizer.tokenize(messages_final[j][i]['content'])))
        sw_length.append(len(tokenizer.tokenize(messages_sw[j][i]['content'])))
        boh_length.append(len(tokenizer.tokenize(messages_boh[j][i]['content'])))
        
print("actual length ", statistics.mean(final_length))
print("boh length ", statistics.mean(boh_length))
print("sw length ", statistics.mean(sw_length))

100%|██████████| 47/47 [00:00<00:00, 362.24it/s]

actual length  56.229268292682924
boh length  23.507317073170732
sw length  18.390243902439025
